##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como migrar salvamento de checkpoints

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/checkpoint_saver"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/checkpoint_saver.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/checkpoint_saver.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/checkpoint_saver.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>


Salvar continuamente o "melhor" modelo ou pesos/parâmetros do modelo traz muitos benefícios. Isto inclui a capacidade de rastrear o progresso do treinamento e carregar modelos salvos de diferentes estados salvos.

No TensorFlow 1, para configurar o salvamento de checkpoints durante o treinamento/validação com as APIs `tf.estimator.Estimator`, especifique um agendamento em `tf.estimator.RunConfig` ou use `tf.estimator.CheckpointSaverHook`. Este guia demonstra como migrar deste fluxo de trabalho para as APIs TensorFlow 2 Keras.

No TensorFlow 2, você pode configurar `tf.keras.callbacks.ModelCheckpoint` de várias maneiras:

- Salvar a "melhor" versão de acordo com uma métrica monitorada usando o parâmetro `save_best_only=True`, onde `monitor` pode ser, por exemplo, `'loss'`, `'val_loss'`, `'accuracy', or` 'val_accuracy'`.
- Salvar continuamente numa certa frequência (usando o argumento `save_freq`).
- Salve apenas os pesos/parâmetros em vez de todo o modelo definindo `save_weights_only` como `True`.

Para mais detalhes, consulte os documentos da API `tf.keras.callbacks.ModelCheckpoint` e a seção *Salve checkpoints durante o treinamento* no tutorial [Como salvar e carregar modelos](../../tutorials/keras/save_and_load.ipynb). Saiba mais sobre o formato Checkpoint na seção *Formato Checkpoint do TF* no guia [Como salvar e carregar modelos Keras](https://www.tensorflow.org/guide/keras/save_and_serialize). Além disso, para incluir tolerância a falhas, você pode usar `tf.keras.callbacks.BackupAndRestore` ou `tf.train.Checkpoint` para aplicar checkpoints manualmente. Saiba mais no [Guia de migração de tolerância a falhas](fault_tolerance.ipynb).

Os [callbacks](https://www.tensorflow.org/guide/keras/custom_callback) do Keras são objetos que são chamados em diferentes pontos durante o treinamento/avaliação/previsão nas APIs integradas Keras `Model.fit`/`Model.evaluate`/`Model.predict`. Saiba mais na seção *Próximas etapas* no final do guia.

## Configuração

Comece com os imports e um dataset simples para fins de demonstração:

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import numpy as np
import tempfile

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## TensorFlow 1: salve checkpoints com APIs tf.estimator

Este exemplo do TensorFlow 1 mostra como configurar `tf.estimator.RunConfig` para salvar checkpoints em cada passo durante o treinamento/avaliação com as APIs `tf.estimator.Estimator`:

In [ ]:
feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

config = tf1.estimator.RunConfig(save_summary_steps=1,
                                 save_checkpoints_steps=1)

path = tempfile.mkdtemp()

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.2,
    model_dir=path,
    config = config
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

In [ ]:
%ls {classifier.model_dir}

## TensorFlow 2: salve checkpoints com um callback Keras para Model.fit

No TensorFlow 2, quando voce usa o Keras `Model.fit` integrado (ou `Model.evaluate`) para treinamento/avaliação, você pode configurar `tf.keras.callbacks.ModelCheckpoint` e passá-lo para o parâmetro `callbacks` de `Model.fit` (ou `Model.evaluate`). (Saiba mais na documentação da API e na seção *Usando callbacks* no guia [Treinamento e avaliação com métodos integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate).)

No exemplo abaixo, você usará um callback `tf.keras.callbacks.ModelCheckpoint` para armazenar checkpoints num diretório temporário:

In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              steps_per_execution=10)

log_dir = tempfile.mkdtemp()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=log_dir)

model.fit(x=x_train,
          y=y_train,
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[model_checkpoint_callback])

In [ ]:
%ls {model_checkpoint_callback.filepath}

## Próximos passos

Saiba mais sobre o uso de checkpoints em:

- Documentação da API: `tf.keras.callbacks.ModelCheckpoint`
- Tutorial: [Como salvar e carregar modelos](../../tutorials/keras/save_and_load.ipynb) (na seção *Salve checkpoints durante o treinamento*)
- Guia: [Salve e carregue modelos Keras](https://www.tensorflow.org/guide/keras/save_and_serialize) (na seção *Formato Checkpoint do TF*)

Saiba mais sobre callbacks em:

- Documentação da API: `tf.keras.callbacks.Callback`
- Guia: [Escrevendo seus próprios callbacks](https://www.tensorflow.org/guide/keras/guide/keras/custom_callback)
- Guia: [Treinamento e avaliação com os métodos integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate) (a seção *Usando callbacks* )

Você talvez também ache úteis os seguintes recursos relacionados à migração:

- O [Guia de migração de tolerância a falhas](fault_tolerance.ipynb): `tf.keras.callbacks.BackupAndRestore` para `Model.fit` ou APIs `tf.train.Checkpoint` e `tf.train.CheckpointManager` para um loop de treinamento personalizado
- O [Guia de migração de paradas antecipadas](early_stopping.ipynb) : `tf.keras.callbacks.EarlyStopping` é um callback integrado para paradas antecipadas (early stopping).
- O [Guia de migração do TensorBoard](tensorboard.ipynb): o TensorBoard permite rastrear e exibir métricas
- O [Guia de migração LoggingTensorHook e StopAtStepHook para callbacks Keras](logging_stop_hook.ipynb)
- O [Guia SessionRunHook para callbacks Keras](sessionrunhook_callback.ipynb)